In [128]:
print("ok")


ok


In [129]:
%pwd

'c:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\research'

In [130]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [131]:
def load_pdf_file(data):
    loader= DirectoryLoader(data, glob= "*.pdf", loader_cls=PyPDFLoader)
    documents= loader.load()

    return documents

In [132]:
extracted_data=load_pdf_file(data=r'C:\Users\aryes\OneDrive\Desktop\Museum chatbot new\Museum-assistance-chatbot\Data')

In [133]:
# extracted_data

In [134]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [135]:
text_chunks=text_split(extracted_data)
print('Length of Text Chunks', len(text_chunks))

Length of Text Chunks 164


In [136]:
# text_chunks

In [137]:
from langchain.embeddings import HuggingFaceEmbeddings

In [138]:
def download_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [139]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully")

Model loaded successfully


In [140]:
embeddings=download_embeddings()

In [141]:
query_result=embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [142]:
# query_result

In [143]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [144]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
HUGGINGFACE_API_KEY=os.environ.get('HUGGINGFACE_API_KEY')

In [145]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec

# pc = Pinecone(api_key=PINECONE_API_KEY)

# index_name= "museumbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

In [146]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["HUGGINGFACE_API_KEY"]=HUGGINGFACE_API_KEY

In [147]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="museumbot",
    embedding=embeddings,
)

In [148]:
from langchain_pinecone import PineconeVectorStore

docsearch= PineconeVectorStore.from_existing_index(
    index_name="museumbot",
    embedding=embeddings
)

docsearch

In [149]:
retreiver=docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3})

In [150]:
retreiver_docs= retreiver.invoke("Where is the Indian Museum Located?")
retreiver_docs

[Document(id='76f28624-7b7a-4d89-8214-2fc57c2bda0e', metadata={'author': 'Sohini Pyne', 'creationdate': '2023-01-30T14:05:14+05:30', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2023-01-30T14:05:14+05:30', 'page': 18.0, 'page_label': '19', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'source': 'C:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\Data\\Indian museum data.pdf', 'title': 'PowerPoint Presentation', 'total_pages': 90.0}, page_content='INDIAN MUSEUM KOLKATA 19'),
 Document(id='26ce7eeb-f9ca-453a-9858-962c1293ecab', metadata={'author': 'Sohini Pyne', 'creationdate': '2023-01-30T14:05:14+05:30', 'creator': 'Microsoft® PowerPoint® for Microsoft 365', 'moddate': '2023-01-30T14:05:14+05:30', 'page': 18.0, 'page_label': '19', 'producer': 'Microsoft® PowerPoint® for Microsoft 365', 'source': 'C:\\Users\\aryes\\OneDrive\\Desktop\\Museum chatbot new\\Museum-assistance-chatbot\\Data\\Indian museum data.pdf', 'title': 'Po

In [151]:
import requests

# HUGGINGFACE_API_KEY = "your_actual_huggingface_api_key"
API_URL = "https://huggingface.co/api/whoami-v2"
headers = {"Authorization": f"Bearer {HUGGINGFACE_API_KEY}"}

response = requests.get(API_URL, headers=headers)
print(response.json())  # Should return your user info if the key is correct


{'error': 'Invalid credentials in Authorization header'}


In [ ]:
# from langchain.llms import HuggingFaceHub


# llm = HuggingFaceHub(
#     repo_id="mistralai/Mistral-7B-v0.1",  # Try this model instead
#     model_kwargs={"temperature": 0.4, "max_length": 500},
#     huggingfacehub_api_token=HUGGINGFACE_API_KEY
# )

# response = llm("When was the Indian Museum established?")
# print(response)


HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/mistralai/Mistral-7B-v0.1 (Request ID: Root=1-67dd7b5d-7af91aab14f1266d03608cb3;074a8c39-46dd-42f9-8c1c-056ef4444abd)

Invalid credentials in Authorization header